In [2]:
import numpy as np

In [3]:
class TreeNode:
    def __init__(self, feature=None, value=None, isleaf=False):
        self.feature = feature
        self.value = value
        self.left = None
        self.right = None
        self.isleaf = isleaf
    
    def tprint(self, depth=0):
        print(f"Depth = {depth}")
        print(f"isleaf = {self.isleaf}")
        print(f"Feature = {self.feature}")
        print(f"Value = {self.value}")
        print("")
        
        if self.left != None:
            self.left.tprint(depth + 1)
        
        if self.right != None:
            self.right.tprint(depth + 1)

In [4]:
class DecisionTree:
    def __init__(self, max_depth=4):
        self.max_depth = max_depth
        self.tree = TreeNode()
    
    def answer(self, Y):
        values, counts = np.unique(Y, return_counts=True)
        max_index = np.argmax(counts)
        return values[max_index]
    
    def predict(self, X):
        res = []
        for x in X:
            current_node = self.tree
            while not current_node.isleaf:
                if x[current_node.feature] <= current_node.value:
                    current_node = current_node.left
                else:
                    current_node = current_node.right
            
            res.append(current_node.value)
        
        return np.array(res)
    
    def __branch(self, X, Y, current_node, current_depth, verbose=False):
        if verbose:
            print(f"Current depth: {current_depth}")
            
        if current_depth > self.max_depth:
            current_node.isleaf = True
            current_node.value = self.answer(Y)
            return
            
        branch_criterie = lambda L, R: len(Y) * self.__H(Y) - len(L) * self.__H(L) - len(R) * self.__H(R)
        
        feature_indexes = np.random.choice(np.arange(X.shape[1]), int(np.sqrt(X.shape[1])), replace=False)
        max_ig = float('-inf')
        max_feature = None
        max_feature_value = None
        
        for i in feature_indexes:
            unique_features_values = np.unique(X[:, i])
            for k in unique_features_values:
                left = Y[X[:, i] <= k]
                right = Y[X[:, i] > k]
                
                ig = branch_criterie(left, right)
                if ig > max_ig:
                    max_ig = ig
                    max_feature = i
                    max_feature_value = k
        
        if max_ig < 1:
            current_node.isleaf = True
            current_node.value = self.answer(Y)
            return
        
        current_node.feature = max_feature
        current_node.value = max_feature_value
        
        current_node.right = TreeNode()
        max_left = X[:, max_feature] <= max_feature_value
        max_right = X[:, max_feature] > max_feature_value
        
        if len(Y[max_left]) > 0:
            current_node.left = TreeNode()
            self.__branch(X[max_left], Y[max_left], current_node.left, current_depth+1)
        if len(Y[max_right]) > 0:
            current_node.right = TreeNode()
            self.__branch(X[max_right], Y[max_right], current_node.right, current_depth+1)
    
    def fit(self, X, Y, verbose=False):
        self.__branch(X, Y, self.tree, current_depth=1, verbose=verbose)
        
        return self
    
    def __H(self, Y):
        values = np.unique(Y, return_counts=True)
        entropy = 0
        for count in values[1]:
            p = count / len(Y)
            entropy -= p * np.log2(p)

        return entropy

In [5]:
tree = DecisionTree(max_depth=15)


In [6]:
tree.answer([2, 2, 5, 4, 4, 4, 1])

4

In [7]:
from sklearn.datasets import load_digits

digits = load_digits()

{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=.3)

In [9]:
tree.fit(X_train, y_train)

In [10]:
Y = np.array([2, 2, 2, 2, 2])

In [11]:
branch_criterie = lambda L, R: len(Y) * __H(Y) - len(L) * __H(L) - len(R) * __H(R)

In [12]:
def __H(Y):
        values = np.unique(Y, return_counts=True)
        entropy = 0
        for count in values[1]:
            p = count / len(Y)
            entropy -= p * np.log2(p)

        return entropy

In [13]:
branch_criterie(np.array([2, 2, 2, 2, 2]), np.array([]))

0.0

In [14]:
tree.tree.tprint(depth=0)

Depth = 0
isleaf = False
Feature = 30
Value = 1.0

Depth = 1
isleaf = False
Feature = 2
Value = 5.0

Depth = 2
isleaf = False
Feature = 28
Value = 9.0

Depth = 3
isleaf = False
Feature = 61
Value = 8.0

Depth = 4
isleaf = False
Feature = 44
Value = 13.0

Depth = 5
isleaf = False
Feature = 34
Value = 8.0

Depth = 6
isleaf = False
Feature = 25
Value = 6.0

Depth = 7
isleaf = False
Feature = 21
Value = 10.0

Depth = 8
isleaf = True
Feature = None
Value = 8

Depth = 8
isleaf = True
Feature = None
Value = 7

Depth = 7
isleaf = False
Feature = 58
Value = 0.0

Depth = 8
isleaf = True
Feature = None
Value = 1

Depth = 8
isleaf = True
Feature = None
Value = 5

Depth = 6
isleaf = False
Feature = 12
Value = 12.0

Depth = 7
isleaf = False
Feature = 19
Value = 7.0

Depth = 8
isleaf = False
Feature = 5
Value = 0.0

Depth = 9
isleaf = True
Feature = None
Value = 6

Depth = 9
isleaf = True
Feature = None
Value = 5

Depth = 8
isleaf = False
Feature = 13
Value = 1.0

Depth = 9
isleaf = True
Feature = No

Value = 9

Depth = 7
isleaf = False
Feature = 62
Value = 0.0

Depth = 8
isleaf = True
Feature = None
Value = 8

Depth = 8
isleaf = True
Feature = None
Value = 2

Depth = 6
isleaf = False
Feature = 4
Value = 12.0

Depth = 7
isleaf = False
Feature = 59
Value = 7.0

Depth = 8
isleaf = True
Feature = None
Value = 7

Depth = 8
isleaf = True
Feature = None
Value = 5

Depth = 7
isleaf = False
Feature = 26
Value = 0.0

Depth = 8
isleaf = True
Feature = None
Value = 3

Depth = 8
isleaf = False
Feature = 52
Value = 8.0

Depth = 9
isleaf = False
Feature = 26
Value = 7.0

Depth = 10
isleaf = True
Feature = None
Value = 7

Depth = 10
isleaf = True
Feature = None
Value = 8

Depth = 9
isleaf = False
Feature = 43
Value = 10.0

Depth = 10
isleaf = True
Feature = None
Value = 8

Depth = 10
isleaf = True
Feature = None
Value = 1

Depth = 1
isleaf = False
Feature = 43
Value = 2.0

Depth = 2
isleaf = False
Feature = 42
Value = 5.0

Depth = 3
isleaf = False
Feature = 10
Value = 8.0

Depth = 4
isleaf = False

In [15]:
predictions = tree.predict(X_test)
predictions

array([7, 4, 9, 7, 2, 8, 0, 8, 5, 8, 6, 5, 7, 9, 2, 3, 5, 9, 4, 6, 5, 3,
       3, 8, 1, 2, 2, 7, 9, 0, 7, 5, 4, 3, 3, 2, 1, 5, 1, 9, 6, 2, 5, 2,
       3, 3, 7, 8, 5, 6, 8, 0, 8, 9, 1, 5, 4, 6, 6, 4, 1, 8, 5, 1, 2, 3,
       6, 9, 1, 7, 1, 0, 5, 8, 2, 7, 1, 0, 4, 9, 6, 2, 1, 0, 9, 6, 9, 8,
       8, 6, 5, 8, 6, 0, 9, 9, 1, 3, 0, 3, 5, 4, 2, 9, 4, 6, 1, 7, 8, 4,
       3, 2, 9, 8, 9, 0, 9, 6, 6, 0, 7, 2, 2, 1, 5, 9, 8, 6, 5, 4, 7, 4,
       6, 8, 9, 8, 1, 6, 3, 6, 8, 0, 5, 8, 3, 0, 4, 2, 1, 6, 2, 7, 2, 2,
       9, 9, 1, 9, 9, 2, 8, 7, 4, 6, 0, 8, 1, 9, 4, 7, 3, 1, 7, 3, 7, 9,
       1, 4, 3, 2, 9, 6, 2, 0, 2, 4, 4, 4, 1, 4, 4, 7, 1, 2, 3, 9, 8, 7,
       5, 5, 2, 5, 4, 1, 6, 7, 7, 6, 9, 1, 4, 0, 7, 5, 6, 3, 4, 9, 8, 8,
       8, 6, 5, 7, 7, 7, 1, 7, 2, 3, 1, 6, 7, 3, 7, 7, 8, 8, 1, 3, 4, 1,
       9, 3, 5, 4, 6, 8, 0, 2, 1, 0, 0, 6, 3, 1, 3, 2, 6, 8, 8, 0, 8, 1,
       8, 3, 0, 1, 4, 2, 4, 2, 2, 2, 0, 8, 4, 4, 2, 7, 1, 1, 6, 5, 8, 1,
       2, 7, 9, 5, 7, 5, 8, 3, 1, 9, 7, 0, 4, 6, 9,

In [16]:
y_test

array([7, 4, 3, 7, 2, 8, 0, 8, 5, 8, 6, 5, 7, 9, 2, 3, 5, 5, 7, 6, 5, 3,
       3, 8, 9, 2, 2, 7, 9, 9, 7, 5, 4, 3, 3, 2, 1, 5, 1, 9, 6, 3, 5, 2,
       3, 3, 7, 8, 5, 6, 8, 0, 9, 9, 8, 5, 4, 2, 6, 4, 1, 8, 5, 5, 2, 3,
       6, 5, 1, 7, 8, 8, 5, 9, 2, 7, 1, 0, 4, 9, 8, 2, 1, 0, 2, 6, 9, 8,
       8, 6, 5, 8, 6, 0, 9, 9, 2, 3, 9, 3, 5, 4, 4, 9, 4, 6, 1, 7, 8, 4,
       3, 9, 5, 8, 9, 0, 9, 6, 6, 0, 7, 2, 2, 1, 5, 9, 2, 6, 6, 4, 3, 5,
       6, 8, 9, 9, 1, 6, 9, 0, 1, 0, 5, 9, 9, 0, 4, 2, 1, 6, 2, 7, 8, 2,
       9, 9, 1, 3, 9, 2, 8, 9, 4, 6, 0, 9, 1, 9, 4, 7, 3, 1, 7, 3, 7, 5,
       8, 4, 3, 2, 9, 6, 2, 0, 2, 4, 4, 4, 1, 4, 4, 7, 1, 2, 3, 2, 2, 7,
       5, 8, 8, 5, 4, 1, 6, 7, 7, 1, 5, 1, 4, 0, 7, 8, 6, 3, 4, 9, 9, 2,
       8, 6, 5, 7, 7, 7, 1, 7, 2, 3, 1, 6, 7, 3, 7, 7, 8, 9, 1, 3, 4, 1,
       9, 3, 5, 4, 6, 8, 0, 2, 1, 0, 0, 9, 3, 1, 3, 2, 6, 8, 8, 0, 0, 1,
       1, 3, 0, 8, 7, 2, 4, 8, 1, 8, 0, 9, 4, 4, 2, 7, 1, 1, 6, 5, 8, 1,
       2, 7, 8, 5, 7, 5, 8, 3, 1, 9, 7, 0, 4, 6, 9,

In [17]:
accuracy = lambda y1, y2: np.count_nonzero(y1 == y2) / len(y2)

In [18]:
accuracy(predictions, y_test)

0.8018518518518518

In [19]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='most_frequent')

dummy.fit(X_train, y_train)

dummy_predictions = dummy.predict(X_test)

accuracy(dummy_predictions[:, np.newaxis], y_test)

48.0

In [20]:
from sklearn.tree import DecisionTreeClassifier

tree_sklearn = DecisionTreeClassifier()

tree_sklearn.fit(X_train, y_train)

sklearn_predictions = tree_sklearn.predict(X_test) 

accuracy(sklearn_predictions[:, np.newaxis], y_test)

54.48518518518519

In [22]:
from load_digits import load_digits

X, y = load_digits()

s
current_number - 4
s
current_number - 8
s
current_number - 5
s
current_number - 2
s
current_number - 9
s
current_number - 7
s
current_number - 6
s
current_number - 1
s
current_number - 3
s


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [24]:
my_tree = DecisionTree(max_depth=30)

In [25]:
my_tree.fit(X_train, y_train, verbose=True)

Current depth: 1


In [26]:
predictions = my_tree.predict(X_test)

In [27]:
accuracy(predictions[:, np.newaxis], y_test)

0.41854838709677417

In [28]:
train_predictions = my_tree.predict(X_train)

In [29]:
accuracy(train_predictions[:, np.newaxis], y_train)

0.9993086761147598